In [78]:
%matplotlib inline
import matplotlib.pyplot as plt
from mocpy import MOC, WCS
import numpy as np
from astropy import units as u
from astropy.table import Table
from astropy.coordinates import Angle, SkyCoord
import ipywidgets as widgets

t = Table.read('2MASS-list-images.fits.gz', format='fits')
t

ra,dec,mjd
deg,deg,
float32,float32,int32
112.29972,-17.323175,51201
112.547806,-16.784193,51201
112.54784,-16.51475,51201
112.052956,-16.784231,51201
112.54784,-16.51475,51201
112.42324,-17.109888,51201
111.928665,-17.379503,51201
111.69493,-18.473375,51215


In [88]:
output = widgets.Output()

def update_plot(change):

    global t, output, time_slider
    output.clear_output(wait=True)
    
    if change is None:
        new_values = time_slider.value
    else:
        new_values = change.new
    selection = np.where((t['mjd']>=new_values[0]) & (t['mjd']<=new_values[1]))
    moc = MOC.from_lonlat(t['ra'][selection], t['dec'][selection], max_norder=7)
    
    

    with output:
        fig = plt.figure(111, figsize=(15, 10))
        
        with WCS(fig, 
                 fov=310 * u.deg,
                 center=SkyCoord(0, 0, unit='deg', frame='galactic'),
                 coordsys="galactic",
                 rotation=Angle(0, u.degree),
                 projection="AIT") as wcs:
            ax = fig.add_subplot(1, 1, 1, projection=wcs)
            # Call fill with a matplotlib axe and the `~astropy.wcs.WCS` wcs object.
            moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="orange")
#            moc.border(ax=ax, wcs=wcs, alpha=0.5, color="black")
        
        plt.xlabel('GLON')
        plt.ylabel('GLAT')
        plt.title('Coverage of 2MASS images between MJD %d and %d' % (new_values[0], new_values[1]))
        plt.grid(color="black", linestyle="dotted")
        plt.show()

In [86]:
min_time = int(np.min(t['mjd']))-1
max_time = int(np.max(t['mjd']))+1

time_slider = widgets.IntRangeSlider(
    value=[min_time, min_time+50],
    min=min_time,
    max=max_time,
    step=1,
    description='MJD interval:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    layout={'width': '90%'}
)

In [87]:
time_slider.observe(update_plot, 'value')
update_plot(None)
widgets.VBox([time_slider, output])



done


VBox(children=(IntRangeSlider(value=(50605, 50655), continuous_update=False, description='MJD interval:', layout=Layout(width='90%'), max=51956, min=50605), Output()))

done
done
done
done
